In [0]:
%run /Workspace/Users/mudashirahul.subhash@hcltech.com/utils/common_functions

### READ NECESSARY FILES FROM SILVER `LAYER`

In [0]:
from datetime import datetime
current_date = datetime.now().strftime("%Y-%m-%d")

In [0]:
dbutils.fs.ls(f"/mnt/bayerhackathon/RAHUL/silver/cleaned_customer_df/{current_date}/")

In [0]:
customer_df1 = spark.read.format("delta").load(f"/mnt/bayerhackathon/RAHUL/silver/cleaned_customer_df/{current_date}/")

In [0]:
customer_df1.display()

In [0]:
customer_behaviour_df1 = spark.read.csv("/mnt/bayerhackathon/customer_behaviour.csv", header=True, inferSchema=True)


### TASK 1 . SPLIT ADDRESS COLUMN IN CUSTOMER FILE TO NUMERIC AND NON NUMERIC PART

In [0]:
splitted_address_customer_df = split_column(customer_df1, "Address")

splitted_address_customer_df.display()


### WRITE FILE TO GOLD LAYER

In [0]:
splitted_address_customer_df.write.format("delta").mode("overwrite").save(f"/mnt/bayerhackathon/RAHUL/gold/splitted_address_customer_df/{current_date}/")

### TASK 2 . APPLY SCD2 LOGIC

In [0]:
customer_scd2_df = spark.read.csv("/mnt/bayerhackathon/customer_SCD2_data.csv", header=True,sep = ";", inferSchema=True)

In [0]:
customer_scd2_df = customer_scd2_df.withColumn("loyalty",lit("none"))

In [0]:
from datetime import datetime
current_date = datetime.now().strftime("%Y-%m-%d")
customer_df = spark.read.format("delta").load(f"/mnt/bayerhackathon/RAHUL/silver/cleaned_customer_df/{current_date}/")

In [0]:
final_scd2_df = apply_scd2(customer_scd2_df,customer_df,"customer_id")

In [0]:
final_scd2_df.orderBy(col("customer_id")).display()

### WRITE DATA TO GOLD LAYER

In [0]:
final_scd2_df.write.format("delta").mode("overwrite").save(f"/mnt/bayerhackathon/RAHUL/gold/final_scd2_df/{current_date}/")

### CUSTOMER 360 VIEW

In [0]:
from datetime import datetime
current_date = datetime.now().strftime("%Y-%m-%d")

In [0]:
cleaned_customer_df = spark.read.format("delta").load(f"/mnt/bayerhackathon/RAHUL/silver/cleaned_customer_df/{current_date}/")
cleaned_order_df = spark.read.format("delta").load(f"/mnt/bayerhackathon/RAHUL/silver/cleaned_order_df/{current_date}/")

cleaned_order_line_df = spark.read.format("delta").load(f"/mnt/bayerhackathon/RAHUL/silver/cleaned_order_line_df/{current_date}/")

cleaned_customer_behaviour_df = spark.read.format("delta").load(f"/mnt/bayerhackathon/RAHUL/silver/cleaned_customer_behaviour_df/{current_date}/") 

In [0]:
cleaned_customer_df.count()

In [0]:
customer_360_view = customer_df1.join(customer_behaviour_df1,on="customer_id",how="left")

In [0]:
customer_360_view.columns

In [0]:
# customer_360_view = customer_360_view.drop(customer_df.state,customer_df.city,customer_df.country,customer_df.zipcode,customer_df.phone,customer_df.Created_date,customer_df.loyalty,customer_df.first_name,customer_df.last_name,customer_df.email,customer_df.gender,customer_df.Address)

In [0]:
customer_360_view.display()

In [0]:
customer_360_view.write.format("delta").mode("overwrite").save(f"/mnt/bayerhackathon/RAHUL/gold/customer_360_view/{current_date}/")

### ORDERS DATA MART

In [0]:

from datetime import datetime
current_date = datetime.now().strftime("%Y-%m-%d")
oders_data_mart_df = spark.read.format("delta").load(f"/mnt/bayerhackathon/RAHUL/silver/updated_purchase_value_df/{current_date}/")

In [0]:
oders_data_mart_df.display()

In [0]:
oders_data_mart_df.write.format("delta").mode("overwrite").save(f"/mnt/bayerhackathon/RAHUL/gold/oders_data_mart_df/{current_date}/")